### Imports

In [18]:
import json

import networkx as nx
import pandas as pd

from utils.utils_go import *

# %load_ext autotime

In [19]:
import pandas as pd
import pingouin as pg
import numpy as np
import seaborn as sn
print(pg.__version__) # 0.5.3
print(pd.__version__) # 2.0.3
print(np.__version__) # 1.24.3
print(sn.__version__) # 0.13.0


0.5.3
2.0.3
1.24.3
0.13.1


### Parameters

In [20]:
file = open("exp.json")
experiment = json.load(file)
exp_num = experiment["exp"]

file = open("output/{}/parameters.json".format(exp_num))
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

data_variations = params["data_variations"]
print("Data variations:", data_variations)

threshold_corr = params["threshold_corr"]
print("Threshold corr:\t", threshold_corr)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

Exp:		 exp65
Data variations: ['none', 'str', 'dyn']
Threshold corr:	 0.3
Groups id:	 ['WT-YNB', 'S51A-YNB', 'WT-Tyr', 'S51A-Tyr', 'WT-total', 'WT-vac', 'avt1-total', 'avt1-vac', 'aa-standard']
Subgroups id:	 {'WT-YNB': ['1', '2'], 'S51A-YNB': ['1', '2'], 'WT-Tyr': ['1', '2'], 'S51A-Tyr': ['1', '2'], 'WT-total': ['1', '2'], 'WT-vac': ['1', '2'], 'avt1-total': ['1', '2'], 'avt1-vac': ['1', '2'], 'aa-standard': ['1', '2']}


### Load dataset

In [21]:
# load dataset groups
df_join_raw = pd.read_csv("input/{}_raw.csv".format(exp), index_col=0)
df_join_raw = df_join_raw.iloc[:, 2:]
df_join_raw

,WT-YNB_1.1,WT-YNB_1.2,WT-YNB_2.1,WT-YNB_2.2,S51A-YNB_1.1,S51A-YNB_1.2,S51A-YNB_2.1,S51A-YNB_2.2,WT-Tyr_1.1,WT-Tyr_1.2,...,avt1-total_2.1,avt1-total_2.2,avt1-vac_1.1,avt1-vac_1.2,avt1-vac_2.1,avt1-vac_2.2,aa-standard_1.1,aa-standard_1.2,aa-standard_2.1,aa-standard_2.2
0,6.508331e+04,5.510899e+04,6.418729e+04,5.865636e+04,7.023975e+04,6.072488e+04,6.418729e+04,7.221445e+04,1.956712e+06,1.956712e+06,...,1.174696e+04,1.207721e+04,1.276583e+04,1.096030e+04,1.233098e+04,1.096030e+04,1.301666e+05,1.197777e+05,1.454335e+05,1.197777e+05
1,3.780118e+06,3.702325e+06,6.536106e+06,6.313462e+06,3.995653e+06,3.601084e+06,3.268053e+06,3.478418e+06,4.919226e+06,6.627347e+06,...,8.928584e+06,1.689391e+07,1.282031e+06,1.813066e+06,4.023445e+06,1.813066e+06,1.171560e+06,9.516028e+05,1.005861e+06,9.516028e+05
2,7.508014e+06,6.226543e+06,5.572912e+06,6.861060e+06,6.269852e+06,5.931642e+06,6.056278e+06,6.627347e+06,1.535205e+06,1.600399e+06,...,2.097152e+06,2.294900e+06,2.767209e+06,2.617939e+06,2.511295e+06,2.617939e+06,4.564192e+05,5.029307e+05,4.692509e+05,5.029307e+05
3,2.294900e+06,2.186209e+06,2.294900e+06,2.082666e+06,2.111739e+06,2.068280e+06,1.970322e+06,1.997827e+06,2.493948e+06,2.408995e+06,...,3.891589e+05,5.503539e+05,5.390277e+05,6.191803e+05,6.321906e+05,6.191803e+05,5.465523e+05,4.288162e+05,4.857990e+05,4.288162e+05
4,4.439384e+05,3.435118e+05,4.408719e+05,5.029307e+05,3.411390e+05,4.170902e+05,3.891589e+05,4.000997e+05,4.199913e+05,5.206665e+05,...,9.128384e+05,1.432397e+06,1.301666e+05,1.569559e+05,4.932393e+04,1.569559e+05,2.301042e+04,2.432243e+04,2.059491e+04,2.432243e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,1.975596e+04,3.511987e+04,4.445321e+04,2.994443e+04,2.643204e+04,3.322542e+04,2.832916e+04,2.317048e+04,4.205530e+04,2.365734e+04,...,5.825119e+04,8.829230e+04,9.255812e+05,1.131653e+06,9.128384e+05,1.131653e+06,1.103654e+04,1.051382e+04,1.015569e+04,1.051382e+04
108,2.957167e+03,5.042768e+03,3.350127e+03,4.299640e+03,3.983995e+03,1.396957e+04,1.029745e+04,1.199379e+04,5.007935e+03,4.973342e+03,...,2.102765e+04,6.330360e+04,5.397486e+04,2.793914e+04,1.895118e+04,2.793914e+04,3.104188e+03,2.936740e+03,3.213656e+03,2.936740e+03
109,3.041062e+08,2.665812e+08,2.135502e+08,2.703026e+08,3.591472e+08,3.148305e+08,3.743987e+08,3.350962e+08,1.552481e+08,1.418706e+08,...,8.435674e+07,8.978680e+07,1.448516e+08,1.296458e+08,1.296458e+08,1.296458e+08,1.541757e+08,1.370379e+08,1.428573e+08,1.370379e+08
110,1.240017e+05,1.021267e+05,1.301666e+05,9.528754e+04,1.189504e+05,9.268190e+04,1.000249e+05,9.661771e+04,1.035523e+05,9.332655e+04,...,2.483350e+04,2.333164e+04,2.266332e+05,3.507297e+05,2.549757e+05,3.507297e+05,2.071046e+05,1.042726e+05,1.869027e+04,1.042726e+05


In [22]:
check_dataset(df_join_raw)

Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 4032


### Generate graphs

In [23]:
""" from sklearn import preprocessing

X_scaled = preprocessing.RobustScaler().fit_transform(df_join_raw)

df_join_raw_log = pd.DataFrame(X_scaled, columns=df_join_raw.columns)
df_join_raw_log """

' from sklearn import preprocessing\n\nX_scaled = preprocessing.RobustScaler().fit_transform(df_join_raw)\n\ndf_join_raw_log = pd.DataFrame(X_scaled, columns=df_join_raw.columns)\ndf_join_raw_log '

In [24]:
# logarithm
""" if has_transformation == "true":
    df_join_raw_log = log10_global(df_join_raw)
else:
    df_join_raw_log = df_join_raw.copy() """

df_join_raw_log = log10_global(df_join_raw)
df_join_raw_log.head()

,WT-YNB_1.1,WT-YNB_1.2,WT-YNB_2.1,WT-YNB_2.2,S51A-YNB_1.1,S51A-YNB_1.2,S51A-YNB_2.1,S51A-YNB_2.2,WT-Tyr_1.1,WT-Tyr_1.2,...,avt1-total_2.1,avt1-total_2.2,avt1-vac_1.1,avt1-vac_1.2,avt1-vac_2.1,avt1-vac_2.2,aa-standard_1.1,aa-standard_1.2,aa-standard_2.1,aa-standard_2.2
0,4.813470,4.741222,4.807449,4.768315,4.846583,4.783367,4.807449,4.858624,6.291527,6.291527,...,4.069926,4.081967,4.106049,4.039823,4.090998,4.039823,5.114500,5.078376,5.162664,5.078376
1,6.577505,6.568475,6.815319,6.800268,6.601588,6.556433,6.514289,6.541382,6.691897,6.821340,...,6.950783,7.227730,6.107899,6.258414,6.604598,6.258414,6.068765,5.978456,6.002538,5.978456
2,6.875525,6.794247,6.746082,6.836391,6.797257,6.773175,6.782206,6.821340,6.186166,6.204228,...,6.321630,6.360764,6.442042,6.417960,6.399898,6.417960,5.659364,5.701508,5.671405,5.701508
3,6.360764,6.339692,6.360764,6.318620,6.324640,6.315609,6.294537,6.300558,6.396887,6.381836,...,5.590127,5.740642,5.731611,5.791817,5.800848,5.791817,5.737632,5.632271,5.686457,5.632271
4,5.647323,5.535942,5.644312,5.701508,5.532931,5.620230,5.590127,5.602168,5.623240,5.716560,...,5.960394,6.156063,5.114500,5.195778,4.693058,5.195778,4.361925,4.386007,4.313760,4.386007


In [25]:
check_dataset(df_join_raw_log)

Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 4032


In [26]:
# split graph in groups and subgroups

list_df_groups_subgroups = split_groups_subgroups(df_join_raw_log, groups_id, subgroups_id)
list_df_groups_subgroups[0][0].head()

,WT-YNB_1.1,WT-YNB_1.2
0,4.813470,4.741222
1,6.577505,6.568475
2,6.875525,6.794247
3,6.360764,6.339692
4,5.647323,5.535942


In [27]:
check_dataset(list_df_groups_subgroups[0][0])

Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 224


In [28]:
# transpose
list_groups_subgroups_t = transpose_global(list_df_groups_subgroups)
list_groups_subgroups_t[0][0]

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
0,4.813470,6.577505,6.875525,6.360764,5.647323,4.386007,6.836391,6.240352,7.134411,5.295118,...,7.324060,5.608189,5.189757,4.705099,4.235492,4.295698,3.470876,8.483025,5.093428,5.502828
1,4.741222,6.568475,6.794247,6.339692,5.535942,4.310750,6.598578,6.273465,7.047112,5.499818,...,7.299977,5.746663,5.346293,4.714130,4.370956,4.545553,3.702669,8.425830,5.009139,5.496808


In [29]:
check_dataset(list_groups_subgroups_t[0][0])

Checking dataset
Count infinite:	 0
Count nan:	 0
Count negative:	 0
Count zero:	 0
Count positive:	 224


In [30]:
# correlation matrix

list_groups_subgroups_t_corr = correlation_global(exp, groups_id, subgroups_id, list_groups_subgroups_t, method="pearson", plot=True)
list_groups_subgroups_t_corr[0][0].head()

0it [00:00, ?it/s]/home/ealvarez/miniconda3/envs/gnn_unsupervised_3.10/lib/python3.10/site-packages/pingouin/correlation.py:954: RuntimeWarning: divide by zero encountered in divide
  D = np.diag(np.sqrt(1 / np.diag(Vi)))
/home/ealvarez/miniconda3/envs/gnn_unsupervised_3.10/lib/python3.10/site-packages/pingouin/correlation.py:955: RuntimeWarning: invalid value encountered in matmul
  pcor = -1 * (D @ Vi @ D)  # Partial correlation matrix
2it [00:00,  8.86it/s]
2it [00:00,  9.49it/s]
2it [00:00,  9.45it/s]
2it [00:00,  9.54it/s]
2it [00:00,  9.50it/s]
2it [00:00,  4.58it/s]
2it [00:00,  9.86it/s]
2it [00:00,  9.83it/s]
2it [00:00,  9.84it/s]
9it [00:02,  4.23it/s]


,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,...,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0
1,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,...,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0
2,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,...,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0
3,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,...,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,1.0,...,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0


<Figure size 640x480 with 0 Axes>

In [31]:
check_dataset(list_groups_subgroups_t_corr[0][0])

Checking dataset
Count infinite:	 0
Count nan:	 442
Count negative:	 6822
Count zero:	 0
Count positive:	 5280


In [32]:
# build graph (corpus graphs)

# list_groups_subgroups_t_corr_g = build_graph_weight_global(exp, list_groups_subgroups_t_corr, groups_id, subgroups_id, threshold=0.5)
list_groups_subgroups_t_corr_g = build_graph_weight_global_directed(exp, list_groups_subgroups_t_corr, groups_id, subgroups_id, threshold=threshold_corr)
# list_groups_subgroups_t_corr_g = build_graph_weight_global_undirected(exp, list_groups_subgroups_t_corr, groups_id, subgroups_id, threshold=threshold_corr)
list_groups_subgroups_t_corr_g[0][0]

100%|██████████| 2/2 [00:00<00:00, 69.55it/s]


WT-YNB 1 110 5995
WT-YNB 2 112 6216


100%|██████████| 2/2 [00:00<00:00, 75.91it/s]


S51A-YNB 1 111 6105
S51A-YNB 2 110 5995


100%|██████████| 2/2 [00:00<00:00, 73.91it/s]


...

 89%|████████▉ | 8/9 [00:00<00:00, 33.62it/s]

avt1-vac 1 112 6216
avt1-vac 2 111 6105


100%|██████████| 9/9 [00:00<00:00, 33.67it/s]

aa-standard 1 112 6216
aa-standard 2 111 6105


,source,target,weight
0,0,1,-1.0
1,0,2,-1.0
2,0,3,-1.0
3,0,4,-1.0
4,0,5,-1.0
...,...,...,...
5990,108,110,1.0
5991,108,111,1.0
5992,109,110,-1.0
5993,109,111,-1.0


In [33]:
# create dataset - nodes/edge data for PyTorch Geometric/DGL framework

for data_variation in data_variations:
    if data_variation == "none":
        create_graph_data_go_features_directed(exp, groups_id, subgroups_id, list_df_groups_subgroups)
        # create_graph_data_go_features_undirected(exp, groups_id, subgroups_id, list_df_groups_subgroups)
    else:
        # dynamic graph to static graph
        create_graph_data_other_go_features(exp, groups_id, subgroups_id, data_variation, list_df_groups_subgroups)

2it [00:00, 45.37it/s]
2it [00:00, 47.83it/s]
2it [00:00, 47.94it/s]
2it [00:00, 46.02it/s]
2it [00:00, 48.20it/s]
2it [00:00, 47.13it/s]
2it [00:00, 47.56it/s]
2it [00:00, 46.82it/s]
2it [00:00, 47.06it/s]
9it [00:00, 22.46it/s]
100%|██████████| 2/2 [00:00<00:00, 88.07it/s]
2it [00:00, 2268.42it/s]
100%|██████████| 2/2 [00:00<00:00, 89.22it/s]
2it [00:00, 2131.25it/s]
100%|██████████| 2/2 [00:00<00:00, 88.05it/s]
2it [00:00, 2231.61it/s]
100%|██████████| 2/2 [00:00<00:00, 86.89it/s]
2it [00:00, 2208.69it/s]
100%|██████████| 2/2 [00:00<00:00, 89.47it/s]
2it [00:00, 2202.89it/s]
100%|██████████| 2/2 [00:00<00:00, 85.01it/s]
2it [00:00, 2082.57it/s]
100%|██████████| 2/2 [00:00<00:00, 86.78it/s]
2it [00:00, 2113.53it/s]
100%|██████████| 2/2 [00:00<00:00, 84.73it/s]
2it [00:00, 2371.67it/s]
100%|██████████| 2/2 [00:00<00:00, 86.14it/s]
2it [00:00, 2385.84it/s]
9it [00:00, 13.77it/s]
100%|██████████| 2/2 [00:00<00:00, 87.92it/s]
2it [00:00, 2296.99it/s]
100%|██████████| 2/2 [00:00<00:00, 86

In [34]:
# details
list_details = []
    
for group in groups_id:
    subgroups = []
    for data_variation in data_variations:
        if data_variation == "none":
            subgroups += subgroups_id[group]
        else:
            subgroups += [data_variation]
    # print(subgroups)
    
    for subgroup in subgroups:
        df_edges = pd.read_csv("output/{}/preprocessing/graphs_data/edges_data_{}_{}.csv".format(exp, group, subgroup))

        G = nx.from_pandas_edgelist(df_edges.iloc[:, [0, 1]])
        list_details.append([group, subgroup, G.number_of_nodes(), G.number_of_edges(), nx.density(G)])

df_details = pd.DataFrame(list_details, columns=["Group", "Subgroup", "Num. nodes", "Num. edges", "Density"])
df_details.to_csv("output/{}/preprocessing/graphs_data/summary.csv".format(exp), index=False)

""" df_details = pd.read_csv("output/{}/preprocessing/graphs_data/summary.csv".format(exp))
df_details """

' df_details = pd.read_csv("output/{}/preprocessing/graphs_data/summary.csv".format(exp))\ndf_details '